In [1]:
from torchDataLoader import FaceDataLoader

In [2]:
from tqdm import tqdm

In [3]:
import torch

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda')

In [6]:
dset_train = FaceDataLoader()

In [7]:
from torch.utils.data import Dataset, DataLoader

In [8]:
train_loader = DataLoader(dset_train, batch_size=40, shuffle=True)

In [9]:
train_loader

In [10]:
from model import EmoModel
model = EmoModel()

In [11]:
model.cuda(device)
model

EmoModel(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU(inplace)
  )
  (fc6): Sequential(
    (0): Dropout(p=0.3)
    (1): Linear(in_features=3200, out_features=3072, bias=True)
    (2): ReLU(inplace)
  )
  (fc8): Sequential(
    (0): Linear(in_features=3072, out_features=6, bias=True)
    (1): Softmax()
  )
)

In [12]:
import torch.optim as optim
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [13]:

num_epochs = 100 # for demonstration purpose I have kept num epochs as 1. For full training make it 10 or 20 or even more if you want to see performance
total_step = len(train_loader)
for epoch in range(num_epochs):
    train_loader = tqdm(train_loader)
    for i, (images, labels) in enumerate(train_loader):
        # CrossEntropy takes target indices not the one hot vector.
        # values, indices = torch.max(labels)
        _, targets = torch.max(labels, -1)
        # Move tensors to the configured device
        images = images.to(device)
        targets = targets.to(device)
        # Forward pass - We will be working on complete batch of 16 images. And not with 1 image at a time
        # remember the images tensor is of shape -> [16, 3, 64, 64],
        # labels tensor is of shape -> [16, 2] and targets tensor is of shape -> [16,]
        outputs = model(images)

        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        """        for i in range(len(images)):
            outputs = model(images[i].unsqueeze_(0))
            loss = criterion(outputs[i], labels[i])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()"""
        
        #if (i+1) % 100 == 0:
        train_loader.set_description('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

  0%|                                                                                                                                                                                       | 0/6 [00:00<?, ?it/s]D:\Anaconda\envs\ml\lib\site-packages\torch\nn\modules\container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
Epoch [36/100], Step [6/6], Loss: 1.6590: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 16.48it/s]


Epoch [74/100], Step [6/6], Loss: 1.7359: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 15.87it/s]


Epoch [100/100], Step [6/6], Loss: 2.0436: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 15.55it/s]
